## Data Understanding and Data Cleansing

In this part, I will try to see:
* what the data is about
* the meaning of each fields, if visible
* the rules of each fields, if visible
* data relevancy
* Role distribution

This is a dataset that contains the first 991 Linkedin job lists under the keywords "Data Science" located in "Indonesia"

In [ ]:
from datetime import datetime
import time
import pandas as pd
import numpy as np

In [ ]:
# Change the file path to where you put the excel file to load
output_file_path = 'C:/Users/Miranti/Documents/_WORK/Portofolio/linkedinwebscraping/DataScience.xlsx'
df = pd.read_excel(output_file_path)

In [ ]:
df.sample(10)

### Split City, Province, Country in Location Field

Comma-separated. There are 3 ways the location is formatted. i.e:
* Jakarta metropolitan area --?> 1 field
* Greater Jakarta --> 1 field
* Greater Semarang --> 1 field
* Badung, Bali, Indonesia --> 3 fields
* Bali, Indonesia --> 2 fields

If only 1 field : [city] / [country]

If 2 fields : [Province, country]

If 3 fields : [city, province, country]

In [ ]:
City = []
Province = []
for row in df['Location']:
    x = row.split(",") #convert each element in the tuple to list
    n_element = len(x) #check how many element in the field (list length)
    #print(n_element) 
    if n_element == 3:
        City.append(x[0]) #take the first element as city
        Province.append(x[1]) #take the second element as province
    elif n_element == 2:
        City.append(np.nan)
        Province.append(x[0])
    else:
        if x[0] == 'Indonesia':
            City.append(np.nan)
            Province.append(np.nan)
        else:
            City.append(x[0])
            Province.append(np.nan)
print('done')

In [ ]:
#Insert new columns after location. location starts from 0.
df.insert(5, 'City', City)
df.insert(6, 'Province', Province)
print('Done')

In [ ]:
df.tail(10)

### Fix Content of Job Level and Type

Turns out, when a job post doesn't have any Job level but has Job type, the content in the field 'Type' is shifted into 'Level'. Upon further inspection, this happened because we find the HTML element by XPATH, which relies on the order of the element. So when the element in the first order is empty, the second element will automatically fill the first one.

I tried to get the element by class name, but turns out there were no distinguishable name.

Thus, for better data accuracy, we need to fix this. But first, we need to define the fields.

#### Field Definition

* Level : the seniority level expected for a job. i.e. (Entry-level, Mid-to-Senior Level, Associate, Director, Executive)
* Type : Employment type of the job, i.e. (Full-time, part-time, internship, temporary, contract)
* Function : The Department or Job function this role falls into
* Industry : The field of the company in general

When there are Type contents in the Level field, we can safely assume that the job does not have seniority Level listed.
For all rows where Level is Full-time/part-time/internship/temporary/contract, we can shift the content into Type field.

In [ ]:
fix_list = ['Full-time', 'Part-time', 'Internship', 'Temporary', 'Contract']
new_level = []
new_type = []
length = len(df['Level'])
df_level = df['Level']
df_type = df['Type']
for row in range(length):
    if str(df_level.iloc[row]) in fix_list:
        new_level.append(np.nan)
        new_type.append(df_level.iloc[row]) #iloc = integer location. Function to select row by its index location
    else:
        new_level.append(df_level.iloc[row])
        new_type.append(df_type.iloc[row])
#Insert new columns after location. location starts from 0.
df['Level'] = new_level
df['Type'] = new_type

In [ ]:
df.head(10)

### Categorize and Remove unrelated Jobs

Even though we used the keywords "Data Scientist", turns out not all job is related to Data Scientist. Some are just General Software Developer, or Python Programmer. Even the "Data Scientist" jobs might come under different role name.

In [ ]:
df['Title'].value_counts()

It might be useful to categorize them into more general job name for the sake of filtering simplicity. So when we want to work with Data Science jobs only, we can easily remove the unrelated jobs more easily later on.

By quickly looking at our data, if the job title contains the words:
* data science
* data scientist
* machine learning
* artificial intelligence
* AI/ML
* ML
* AI Engineer

we can categorize them into Data Scientist. Otherwise, we will mark them as 'Others'

In [ ]:
df['Title'] = df['Title'].str.lower() #convert all values in Title to lowercase
keyword_list= ['data science', 'data scientist', 'machine learning', 'artificial intelligence', 'ai/ml', 'ml', 'ai engineer']
swe_list = ['software','software engineer', 'programmer', 'full stack', 'application', 'developer']        
fe_list = ['front-end', 'front end', 'frontend', 'frontend developer']
be_list = ['back end', 'back-end', 'backend developer']
length = len(df['Title'])
df_title = df['Title']
title_category = []
for row in range(length):
    if any(element in str(df_title.iloc[row]) for element in keyword_list)==True:
        title_category.append('Data Science')
    elif any(element in str(df_title.iloc[row]) for element in fe_list)==True:
        title_category.append('Front-End Engineer')
    elif any(element in str(df_title.iloc[row]) for element in be_list)==True:
        title_category.append('Back-End Engineer')
    elif any(element in str(df_title.iloc[row]) for element in swe_list)==True:
        title_category.append('Software Engineer')
    else:
        title_category.append('Others')

#Insert new columns after location. location starts from 0.
df['Title Category'] = title_category

In [ ]:
df.sample(10)

In [ ]:
# Count the total records by title category
df['Title Category'].value_counts()

In [ ]:
# Slice only job lists with title category = Data Science
df_ds = df[df['Title Category']=='Data Science']

In [ ]:
# Slice only job lists with title category = Software Engineer
# This is just a sample, for checking purpose
df_swe = df[df['Title Category']=='Software Engineer']

In [ ]:
len(df_ds)

In [ ]:
df_swe.sample(10)

### Select important column for Data Analysis

In [ ]:
df.drop('Description', inplace=True, axis=1)
df.drop('ID', inplace=True, axis=1)
df_clean = df
df_clean.columns = ['date', 'company', 'title', 'location','city','province','level','type','function','industry','title_category']

In [ ]:
df_clean.head(10)

### Export to excel or csv

In [ ]:
output_file_path_excel = 'C:/Users/Miranti/Documents/_WORK/Portofolio/linkedinwebscraping/DataScienceProcessed.xlsx'
output_file_path_csv = 'C:/Users/Miranti/Documents/_WORK/Portofolio/linkedinwebscraping/DataScienceProcessed.csv'
df_ds.to_excel(output_file_path_excel, index=False)
df.to_csv(output_file_path_csv, sep='$', index=False) #csv separator used = '$'